In [1]:
import os
import sys
scriptPath = os.path.abspath('')
parentPath = os.path.join(scriptPath, '..')
sys.path.append(parentPath)

In [2]:
from db import DB
import plotly.express as pex
import plotly.graph_objects as go
import pandas as pd

In [3]:
db = DB()
file = db.fs.getOpt(op='LastDB')
db.openDB(file)

opened db: /home/mi/Dropbox/prog/python/banking/dev/integrationTest/fixtures/db/testDB.s3db


True

In [4]:
startDate, endDate = db.dataRange()
# ['2021-09-01', '2022-01-17']

In [5]:
selCat = 'Grandpa'
ignoreCat = ['wlasne','dochody']

kidsCats = db.tree.flatChild(catStart=selCat, ignoreCat=ignoreCat)

In [7]:
# categories
df = db.op.get(category=list(kidsCats.keys()), startDate=startDate, endDate=endDate, cols=[db.DATA_OP, db.AMOUNT, db.CATEGORY])

df.loc[:,'parCat'] = [kidsCats[i] for i in df.loc[:,db.CATEGORY]]


In [8]:
# bilans
catBilans = list(db.tree.allChild().keys())
[catBilans.remove(c) for c in ignoreCat]
df_bilans = db.op.get(category=catBilans, startDate=startDate, endDate=endDate, cols=[db.DATA_OP, db.AMOUNT, db.CATEGORY])

df_bilans.loc[:,'parCat'] = df_bilans.loc[:,db.CATEGORY]

In [9]:
# group
timeGrp = pd.Grouper(key=db.DATA_OP, freq='M')
df = df.groupby([timeGrp, 'parCat']).sum()
df_bilans = df_bilans.groupby(timeGrp).sum()
df.reset_index(inplace=True)
df_bilans.reset_index(inplace=True)

In [10]:
fig = go.Figure()
fig.add_traces(list(pex.bar(df,
                           x=db.DATA_OP,
                           y=db.AMOUNT,
                           color='parCat',
                           barmode='group').select_traces()))

fig.add_traces(list(pex.line(df_bilans,
                               x=db.DATA_OP,
                               y=db.AMOUNT).select_traces()))

fig.show()
